# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
df.shape

(68817, 86)

# Split the Data into Training and Testing

In [6]:
# Create our features
X = df.drop(columns='loan_status')
X = pd.get_dummies(X)

In [7]:
# Create our target
y = df.loc[:, target].copy()

In [8]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [9]:
# Check the balance of our target values
y["loan_status"].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [10]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)
X_train.shape

(51612, 95)

In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(51612, 95)
(17205, 95)
(51612, 1)
(17205, 1)


In [12]:
y.shape

(68817, 1)

In [13]:
X.shape

(68817, 95)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [14]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled_1, y_resampled_1 = ros.fit_resample(X_train, y_train)

In [15]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
ros_MODEL = LogisticRegression(solver='lbfgs', random_state=1)
ros_MODEL.fit(X_resampled_1, y_resampled_1)

LogisticRegression(random_state=1)

In [16]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_prediction_1 = ros_MODEL.predict(X_test)
balanced_accuracy_score(y_test, y_prediction_1)

0.6742571941946299

In [17]:
y_test

,loan_status
7034,low_risk
38804,low_risk
65533,low_risk
37736,low_risk
6390,low_risk
...,...
31854,low_risk
27835,low_risk
3089,low_risk
43343,low_risk


In [18]:
X_test

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
7034,10000.0,0.1308,337.33,65000.0,33.30,2.0,0.0,7.0,0.0,8474.0,...,1,1,0,1,0,1,1,0,1,1
38804,10275.0,0.1614,250.64,107000.0,17.05,0.0,1.0,17.0,1.0,16893.0,...,0,1,0,1,1,0,1,0,1,1
65533,40000.0,0.0819,1256.97,170000.0,14.33,0.0,0.0,11.0,0.0,27265.0,...,0,1,0,1,0,1,1,0,1,1
37736,9000.0,0.1356,305.68,98000.0,17.17,0.0,1.0,9.0,0.0,12609.0,...,0,1,0,1,1,0,1,0,1,1
6390,9100.0,0.1640,321.73,90000.0,20.92,0.0,1.0,22.0,0.0,13075.0,...,1,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31854,20000.0,0.2235,556.37,62000.0,18.91,0.0,2.0,5.0,0.0,10845.0,...,0,1,0,1,0,1,1,0,1,1
27835,20000.0,0.0702,617.73,90000.0,29.73,0.0,0.0,10.0,0.0,33966.0,...,0,1,0,1,0,1,1,0,1,1
3089,15000.0,0.1557,361.36,50000.0,17.23,0.0,0.0,8.0,0.0,8266.0,...,1,1,0,1,0,1,1,0,1,1
43343,5600.0,0.1131,184.17,50000.0,0.84,1.0,0.0,3.0,0.0,736.0,...,0,1,1,0,1,0,1,0,1,1


In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_prediction_1)

array([[   75,    26],
       [ 6740, 10364]])

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_prediction_1))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.74      0.61      0.02      0.67      0.46       101
   low_risk       1.00      0.61      0.74      0.75      0.67      0.44     17104

avg / total       0.99      0.61      0.74      0.75      0.67      0.44     17205



### SMOTE Oversampling

In [21]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled_2, y_resampled_2 = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)

In [22]:
# Train the Logistic Regression model using the resampled data
SMOTE_MODEL = LogisticRegression(solver='lbfgs', random_state=1)
SMOTE_MODEL.fit(X_resampled_2, y_resampled_2)

LogisticRegression(random_state=1)

In [24]:
# Calculated the balanced accuracy score
y_prediction_2 = SMOTE_MODEL.predict(X_test)
balanced_accuracy_score(y_test, y_prediction_2)

0.6623356588465208

In [25]:
y_prediction_2

array(['low_risk', 'high_risk', 'low_risk', ..., 'high_risk', 'low_risk',
       'low_risk'], dtype=object)

In [26]:
y_test

,loan_status
7034,low_risk
38804,low_risk
65533,low_risk
37736,low_risk
6390,low_risk
...,...
31854,low_risk
27835,low_risk
3089,low_risk
43343,low_risk


In [27]:
X_test

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
7034,10000.0,0.1308,337.33,65000.0,33.30,2.0,0.0,7.0,0.0,8474.0,...,1,1,0,1,0,1,1,0,1,1
38804,10275.0,0.1614,250.64,107000.0,17.05,0.0,1.0,17.0,1.0,16893.0,...,0,1,0,1,1,0,1,0,1,1
65533,40000.0,0.0819,1256.97,170000.0,14.33,0.0,0.0,11.0,0.0,27265.0,...,0,1,0,1,0,1,1,0,1,1
37736,9000.0,0.1356,305.68,98000.0,17.17,0.0,1.0,9.0,0.0,12609.0,...,0,1,0,1,1,0,1,0,1,1
6390,9100.0,0.1640,321.73,90000.0,20.92,0.0,1.0,22.0,0.0,13075.0,...,1,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31854,20000.0,0.2235,556.37,62000.0,18.91,0.0,2.0,5.0,0.0,10845.0,...,0,1,0,1,0,1,1,0,1,1
27835,20000.0,0.0702,617.73,90000.0,29.73,0.0,0.0,10.0,0.0,33966.0,...,0,1,0,1,0,1,1,0,1,1
3089,15000.0,0.1557,361.36,50000.0,17.23,0.0,0.0,8.0,0.0,8266.0,...,1,1,0,1,0,1,1,0,1,1
43343,5600.0,0.1131,184.17,50000.0,0.84,1.0,0.0,3.0,0.0,736.0,...,0,1,1,0,1,0,1,0,1,1


In [28]:
# Display the confusion matrix
confusion_matrix(y_test, y_prediction_2)

array([[   64,    37],
       [ 5285, 11819]])

In [29]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_prediction_2))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.63      0.69      0.02      0.66      0.44       101
   low_risk       1.00      0.69      0.63      0.82      0.66      0.44     17104

avg / total       0.99      0.69      0.63      0.81      0.66      0.44     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [30]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled_3, y_resampled_3 = cc.fit_resample(X_train, y_train)

In [31]:
X_resampled_3

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,15000.000000,0.189400,388.620000,92000.000000,18.370000,0.000000,0.000000,13.000000,0.000000e+00,15745.000000,...,0,1,0,1,0,1,1,0,1,1
1,35000.000000,0.150200,833.020000,56800.000000,45.510000,0.000000,0.000000,11.000000,0.000000e+00,21458.000000,...,0,1,0,1,0,1,0,1,1,1
2,15000.000000,0.075600,467.010000,39272.000000,42.360000,0.000000,0.000000,8.000000,1.000000e+00,9913.000000,...,1,1,0,1,0,1,0,1,1,1
3,3600.000000,0.075600,112.090000,50000.000000,13.350000,0.000000,0.000000,5.000000,0.000000e+00,1129.000000,...,1,1,0,1,0,1,1,0,1,1
4,28500.000000,0.118000,631.100000,89000.000000,11.480000,0.000000,1.000000,8.000000,0.000000e+00,101.000000,...,0,1,0,1,1,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,30000.000000,0.075600,934.020000,800000.000000,9.850000,0.000000,0.000000,18.000000,0.000000e+00,360411.000000,...,0,1,0,1,0,1,1,0,1,1
488,24151.266667,0.118735,682.886933,81154.010667,22.863360,0.112000,0.458667,14.869333,7.733333e-02,29467.154667,...,0,1,0,0,0,0,0,0,1,1
489,18786.111111,0.119211,576.273333,158334.222222,20.496667,0.555556,0.333333,21.000000,2.222222e-01,22804.444444,...,0,1,0,0,0,0,1,0,1,1
490,27493.333333,0.108547,831.430667,476041.666667,13.749333,0.200000,0.800000,15.666667,-2.775558e-17,45284.066667,...,0,1,0,0,0,0,0,0,1,1


In [32]:
y_resampled_3

,loan_status
0,high_risk
1,high_risk
2,high_risk
3,high_risk
4,high_risk
...,...
487,low_risk
488,low_risk
489,low_risk
490,low_risk


In [33]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
cc_MODEL = LogisticRegression(solver='lbfgs', random_state=1)
cc_MODEL.fit(X_resampled_3, y_resampled_3)

LogisticRegression(random_state=1)

In [34]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_prediction_3 = cc_MODEL.predict(X_test)
balanced_accuracy_score(y_test, y_prediction_3)

0.5443043836656818

In [35]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_prediction_3)

array([[   68,    33],
       [10000,  7104]])

In [36]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_prediction_3))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.67      0.42      0.01      0.53      0.29       101
   low_risk       1.00      0.42      0.67      0.59      0.53      0.27     17104

avg / total       0.99      0.42      0.67      0.58      0.53      0.27     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [37]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smtn = SMOTEENN(random_state=1)
X_resampled_4, y_resampled_4 = smtn.fit_resample(X, y)

In [38]:
X_resampled_4

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,4400.0,0.2000,163.52,41000.0,31.82,0.0,0.0,10.0,0.0,5912.0,...,1,1,0,1,0,1,1,0,1,1
1,5000.0,0.2880,208.99,80000.0,16.88,0.0,2.0,13.0,0.0,110328.0,...,1,1,0,1,0,1,1,0,1,1
2,5400.0,0.0819,169.70,58000.0,36.50,0.0,2.0,42.0,0.0,31039.0,...,1,1,1,0,0,1,1,0,1,1
3,11000.0,0.1474,379.92,80000.0,19.82,0.0,1.0,15.0,0.0,15662.0,...,1,1,0,1,0,1,1,0,1,1
4,1000.0,0.1797,36.14,21456.0,16.72,1.0,0.0,6.0,0.0,1348.0,...,1,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130475,10000.0,0.1502,346.76,26000.0,9.60,0.0,0.0,9.0,0.0,2684.0,...,0,1,0,1,0,1,1,0,1,1
130476,12000.0,0.2727,368.37,63000.0,29.07,0.0,0.0,8.0,0.0,13314.0,...,0,1,0,1,0,1,1,0,1,1
130477,5000.0,0.1992,185.62,52000.0,14.86,0.0,0.0,5.0,1.0,3715.0,...,0,1,0,1,0,1,1,0,1,1
130478,40000.0,0.0646,1225.24,520000.0,9.96,0.0,1.0,21.0,0.0,59529.0,...,0,1,1,0,0,1,1,0,1,1


In [39]:
y_resampled_4

,loan_status
0,high_risk
1,high_risk
2,high_risk
3,high_risk
4,high_risk
...,...
130475,low_risk
130476,low_risk
130477,low_risk
130478,low_risk


In [40]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
smtn_MODEL = LogisticRegression(solver='lbfgs', random_state=1)
smtn_MODEL.fit(X_resampled_4, y_resampled_4)

LogisticRegression(random_state=1)

In [41]:
# Calculated the balanced accuracy score
y_prediction_4 = smtn_MODEL.predict(X_test)
balanced_accuracy_score(y_test, y_prediction_4)

0.6615015073771175

In [42]:
y_test

,loan_status
7034,low_risk
38804,low_risk
65533,low_risk
37736,low_risk
6390,low_risk
...,...
31854,low_risk
27835,low_risk
3089,low_risk
43343,low_risk


In [43]:
# Display the confusion matrix
confusion_matrix(y_test, y_prediction_4)

array([[   74,    27],
       [ 7007, 10097]])

In [44]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_prediction_4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.73      0.59      0.02      0.66      0.44       101
   low_risk       1.00      0.59      0.73      0.74      0.66      0.43     17104

avg / total       0.99      0.59      0.73      0.74      0.66      0.43     17205

